<h1> Housing Median Price Predictor </h1>

California census data includes metrics such as the population, median income, and median housing price for each district.

This uses metrics data to build a model to predict the median housing price in any district in the state.

<h2 style="background-color:rgba(100,100,100,0.5);"> Problem Summary </h2>

Housing price predictor is a typical supervised learning task, since we have labeled training examples (each instance comes with the expected output, i.e., the district’s median housing price).

It is also a typical multiple regression problem since the system will use multiple features to make a prediction for the output.

It is also a univariate regression problem, since we are only trying to predict a single value for each district.

<h2 style="background-color:rgba(100,100,100,0.5);"> Fetch and load data </h2>

In [1]:
import os
import tarfile
import urllib

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    os.makedirs(housing_path, exist_ok=True)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

In [2]:
import pandas as pd

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

ModuleNotFoundError: No module named 'pandas'

<h2 style="background-color:rgba(100,100,100,0.5);"> Analyze the data

In [ ]:
fetch_housing_data()
housing = load_housing_data()

# Display head
housing.head()

: 

In [ ]:
# Display info
housing.info()

# Display percentage of missing values for each attribute
print(housing.isnull().sum() / len(housing) * 100)

: 

In this dataset, there are 20,640 instances, which means that it is fairly small by Machine Learning standards.

Data cleaning will need to be done to address some missing values (e.g. total bedrooms).

The total_bedrooms attribute has only 20,433 nonnull values, meaning that 207 districts are missing this feature (about 1%).

All attributes are numerical, except the ocean_proximity field. The values in the ocean_proximity column are probably a categorical attribute and needs to be mapped to numerical values (encoding).

In [ ]:
housing.describe()

: 

<h3><b>Preview the center and dispersion of the features</b></h3>

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
housing.hist(bins=50, figsize=(20, 15))
plt.show()

: 

The plot for the distributions of the features are shown above.

<h2 style="background-color:rgba(100,100,100,0.5);"> Sample the data for testing and training purposes</h2>

One way of splitting data for testing and training is to pick some instances randomly, typically 20% for testing purposes:

In [ ]:
import numpy as np

def split_train_test(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

: 

Scikit-Learn provides a few functions to split datasets into multiple subsets in various ways. The simplest function is train_test_split(), which uses a random_state parameter that allows you to set the random generator seed. Second, you can pass it multiple datasets with an identical number of rows, and it will split them on the same indices.

In [ ]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

: 

<h3><b>Stratified Sampling</b></h3>

The median income is a very important attribute to predict median housing prices. We want to ensure that the test set is representative of the various categories of incomes.

Since the median income is a continuous numerical attribute, you first need to create an income category attribute.

The following code uses the pd.cut() function to create an income category attribute with five categories (labeled from 1 to 5): category 1 ranges from 0 to 1.5 (i.e., less than $15,000), category 2 from 1.5 to 3, and so on.

In [ ]:
housing["income_cat"] = pd.cut(housing["median_income"],
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5])
housing["income_cat"].hist()

: 

Now we can do stratified sampling based on the income category using Scikit-Learn’s StratifiedShuffleSplit class:

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

: 

In [ ]:
# print income category proportions
# Note: in jupyter you can use the print command or the >>>, they are equivalent
# But, if you invoke >>> it has to be the only command, you can't have comments also in the cell
print(strat_test_set["income_cat"].value_counts() / len(strat_test_set))

: 

In [ ]:
# Now that we have completed stratified sampling, delete the income_cat column, so that the data is restored to its original state

for set_ in (strat_test_set, strat_train_set):
    set_.drop("income_cat", axis=1, inplace=True)

: 

<h2 style="background-color:rgba(100,100,100,0.5);"> Discover and visualize the data to gain insights </h2>

Making a copy of training set for analysis purposes:

In [ ]:
housing = strat_train_set.copy()

: 

<h3><b>Visualizing Geographical Data</b></h3>

In [ ]:
# simple scatter plot
housing.plot(kind="scatter", x="longitude", y="latitude")

# using alpha option to view densities
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.1)

# add housing prices to plot
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.4,
    s=housing["population"]/100, label="population", figsize=(10,7),
    c="median_house_value", cmap=plt.get_cmap("jet"), colorbar=True,
)
plt.legend()

: 

<h3><b>Looking for correlations</b></h3>

In [ ]:
# Since dataset is not too large, standard correlation coefficient (Pearsons' r) can be computed for all attribues
corr_matrix = housing.corr()

corr_matrix["median_house_value"].sort_values(ascending=False)

: 

The correlation coefficient ranges from –1 to 1.

When it is close to 1, it means that there is a strong positive correlation; for example, the median house value tends to go up when the median income goes up.

When the coefficient is close to –1, it means that there is a strong negative correlation; you can see a small negative correlation between the latitude and the median house value (i.e., prices have a slight tendency to go down when you go north).

Finally, coefficients close to 0 mean that there is no linear correlation.

The correlation coefficient only measures linear correlations.

<h3><b>Scatter matrix to view correlations</b></h3>

In [ ]:
from pandas.plotting import scatter_matrix

attributes = ["median_house_value", "median_income", "total_rooms",
              "housing_median_age"]
scatter_matrix(housing[attributes], figsize=(12, 8))

: 

In [ ]:
housing.plot(kind="scatter", x="median_income", y="median_house_value", alpha=0.1)

: 

<h2 style="background-color:rgba(100,100,100,0.5);"> Prepare the Data for Machine Learning Algorithms </h2>

<h3><b> SCIKIT-LEARN </b></h3>

Prepare the data for Machine Learning algorithms. But first let’s revert to a clean training set (by copying strat_train_set once again). Let’s also separate the predictors and the labels:

In [ ]:
housing = strat_train_set.drop("median_house_value", axis=1)
housing_labels = strat_train_set["median_house_value"].copy()

: 

<h3><b> Data Cleaning - Missing attributes </b></h3>

We saw earlier that the total_bedrooms attribute has some missing values. To fix this, we have three options:

<ol type="1">
    <li>Get rid of the corresponding districts</li>
    <li>Get rid of the whole attribute</li>
    <li>Set the values to some value (zero, the mean, the median etc.)</li>
</ol>

DataFrame’s dropna(), drop(), and fillna() methods allow to do this:

In [ ]:
housing.dropna(subset=["total_bedrooms"])    # option 1
housing.drop("total_bedrooms", axis=1)       # option 2
median = housing["total_bedrooms"].median()  # option 3
housing["total_bedrooms"].fillna(median, inplace=True)

: 

Scikit-Learn provides a handy class to take care of missing values: SimpleImputer. Create a SimpleImputer instance, specifying to replace each attribute’s missing values with the median of that attribute:

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")

# Since the median can only be computed on numerical attributes, you need to create 
# a copy of the data without the text attribute ocean_proximity:
housing_num = housing.drop("ocean_proximity", axis=1)

imputer.fit(housing_num)

# The imputer has simply computed the median of each attribute and stored the result 
# in its statistics_ instance variable.
# Apply the imputer to all the numerical attributes:
print(imputer.statistics_)
print(housing_num.median().values)

# The “trained” imputer can be used to transform the training set by replacing
# missing values with the learned medians:
X = imputer.transform(housing_num)

# Put the imputed values back into a pandas DataFrame:
housing_tr = pd.DataFrame(X, columns=housing_num.columns, index=housing_num.index)

: 

<h3><b> Data Cleaning - Handling text and categorical attributes </b></h3>

Scikit-Learn’s OrdinalEncoder class can be used to convert text to numbers.

In [ ]:
# In this dataset, there is just one categorical attribute: the ocean_proximity attribute.
# Let’s look at its value for the first 10 instances:

housing_cat = housing[["ocean_proximity"]]
housing_cat.head(10)

: 

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)
print(housing_cat_encoded[:10])

# categories_ attribute is a list containing a 1D array of categories for each categorical attribute
# (in this case, a list containing a single array since there is just one categorical attribute)
print(ordinal_encoder.categories_)

: 

One-hot encoding

Scikit-Learn provides a OneHotEncoder class to convert categorical values into one-hot vectors.

In [ ]:
from sklearn.preprocessing import OneHotEncoder
cat_encoder = OneHotEncoder()
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)

print(cat_encoder.categories_)

: 

The output is a SciPy sparse matrix, instead of a NumPy array. You can use it mostly like a normal 2D array, but if you really want to convert it to a dense NumPy array, just call the toarray() method:

In [ ]:
housing_cat_1hot.toarray()

: 

<h3><b> Feature Scaling </b></h3>

There are two common ways to get all attributes to have the same scale: min-max scaling and standardization.

Min-max scaling (also called normalization)
>values are shifted and rescaled so that they end up ranging from 0 to 1. This can be computed by subtracting the min value and dividing by the max minus the min. Scikit-Learn provides a transformer called MinMaxScaler for this.

Standardization
>First subtract the mean value so standardized values always have a zero mean, then divide by the standard deviation so that the resulting distribution has unit variance. Scikit-Learn provides a transformer called StandardScaler for standardization.

<h3><b> Custom Transformers </b></h3>

Create a custom transformer class by adding BaseEstimator and TransformerMixin as base classes. The two extra methods (get_params() and set_params()) will be useful for automatic hyperparameter tuning.

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room=True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self # nothing else to do
    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)
# Note dataframe.values returns a numpy array, the recommendation is to use to_numpy()

: 

<h3><b> Transformation Pipelines </b></h3>

Scikit-Learn provides the Pipeline class to help with sequences of transformations.

Pipeline for numerical attributes:

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ('attribs_adder', CombinedAttributesAdder()),
    ('std_scaler', StandardScaler())
])

housing_num_tr = num_pipeline.fit_transform(housing_num)

: 

<h3><b> Single Transformer Composition </b></h3>

Scikit-Learn has the ColumnTransformer for this purpose.

In [ ]:
from sklearn.compose import ColumnTransformer

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", OneHotEncoder(), cat_attribs),
])

housing_prepared = full_pipeline.fit_transform(housing)

: 

<h2 style="background-color:rgba(100,100,100,0.5);"> Select and Train Models </h2>

Select a model that you think might work best first. Train and evaluate the performance of this model. The model might fit correctly, underfit or overfit the actual data.

The general approach is to try out models from various categories of Machine Learning algorithms to see which one best fits the data.

<h3><b> Training and Evaluating on the Training Set </b></h3>

Select a model to train and evaluate on the data set. Linear Regression model - let's use this first to evaluate on the training set.

<h3><b> Predict values for some data inputs </b></h3>

In [ ]:
# Linear Regression Model
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]

some_data_prepared = full_pipeline.transform(some_data)
print("Predictions:", lin_reg.predict(some_data_prepared))
print("Labels: ", list(some_labels))

: 

<h3><b> Measure the performance of the model on the whole training set </b></h3>

Evaluate the model's RMSE on the whole training set

In [ ]:
from sklearn.metrics import mean_squared_error

housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)

print(lin_rmse)

: 

<h3><b> Solutions for underfitting </b></h3>

The model is underfitting the training data when the model performs poorly on the training data. The model is unable to capture the relationship between the input examples and the target values.

The main ways to fix underfitting are:
>Select a more powerful model

>Feed the training algorithm with better features

>Reduce the constraints on the model (increase degrees of freedom)

<h3><b> Try other models and measure scores </b></h3>

So, let’s try a more complex model (Decision Tree) to see how it does.

In [ ]:
# Decision Tree Regression Model
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, housing_labels)

housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)

print(tree_rmse)

: 

<h3><b> Better evaluation using Cross-Validation </b></h3>

Scikit-Learn’s K-fold provides cross-validation feature. randomly split the training set into distinct subsets called, then train and evaluates the Decision Tree model on the subsets, picking a different subset for evaluation every time and training on the other subsets.

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(tree_reg, housing_prepared, housing_labels,
                         scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

display_scores(tree_rmse_scores)

: 

Now the Decision Tree results looks worse than the Linear Regression model.

Just to compare, lets get the cross validation scores for the Linear Regression Model:

In [ ]:
lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels,
                                scoring="neg_mean_squared_error", cv=10)

lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

: 

The Decision Tree model is overfitting so badly that it performs worse than the Linear Regression model.

<h3><b> Solutions for overfitting </b></h3>

The model is overfitting your training data when you see that the model performs well on the training data but does not perform well on the evaluation data. This is because the model is memorizing the data it has seen and is unable to generalize to unseen examples.

The main ways to fix overfitting are:

>Select a more simplified model

>Get a lot more training data

>Increase the constraints on the model (reduce the degrees of freedom - regularization)

<h3><b> Continue to try any other models and measure scores </b></h3>

Let's try RandomForestRegressor model, this is an Ensemble model.

In [ ]:
# Random Forest Regression Model
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor()
forest_reg.fit(housing_prepared, housing_labels)

housing_predictions = forest_reg.predict(housing_prepared)
forest_mse = mean_squared_error(housing_labels, housing_predictions)
forest_rmse = np.sqrt(forest_mse)

forest_scores = cross_val_score(forest_reg, housing_prepared, housing_labels,
                         scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)

display_scores(forest_rmse_scores)

: 

Random Forests look very promising, but the model is still overfitting the training set.

<h3><b> Save model and data </b></h3>

Save every model you experiment with so that you can come back easily to any model you want.

Save both the hyperparameters and the trained parameters, as well as the cross-validation scores and perhaps the actual predictions as well.

In [ ]:
# Dump and load data

# import joblib

# joblib.dump(my_model, "my_model.pkl")
# and later.....
# my_model_loaded = joblib.load("my_model.pkl")

: 

<h2 style="background-color:rgba(100,100,100,0.5);"> Fine-Tune Your Model </h2>

After shortlisting promising models, we need to fine-tune the models.

<h3><b> Grid Search </b></h3>

Scikit-Learn’s GridSearchCV can be used. All you need to do is tell it which hyperparameters you want it to experiment with and what values to try out, and it will use cross-validation to evaluate all the possible combinations of hyperparameter values. The get_params() method to provides the list of hyperparameters that the model uses.

For example, the following code searches for the best combination of hyperparameter values for the RandomForestRegressor:

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
  ]

forest_reg = RandomForestRegressor()

grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

grid_search.fit(housing_prepared, housing_labels)

: 

In [ ]:
# get best combination of parameters
print(grid_search.best_params_)

# get best estimator
grid_search.best_estimator_

# get evaluation scores
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

: 

<h3><b> Randomized Search </b></h3>

The grid search approach works when you are exploring relatively few combinations, but when the hyperparameters are many, we can use RandomizedSearchCV instead.

<h3><b> Ensemble Methods </b></h3>

Another way to fine-tune your system is to try to combine the models that perform best. This is called an ensemble model and likely perform better than any particular model, especially if the individual models have different types of errors.

<h2 style="background-color:rgba(100,100,100,0.5);"> Analyze the Best Models and Their Errors </h2>

We can gain good insights on the problem by inspecting the best models.

For example, the RandomForestRegressor can indicate the relative importance of each attribute for making accurate predictions:

In [ ]:
feature_importances =  grid_search.best_estimator_.feature_importances_
print(feature_importances)

extra_attribs = ["rooms_per_hhold", "pop_per_hhold", "bedrooms_per_room"]
cat_encoder = full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = list(cat_encoder.categories_[0])
attributes = num_attribs + extra_attribs + cat_one_hot_attribs
print(sorted(zip(feature_importances, attributes), reverse=True))

: 

<h2 style="background-color:rgba(100,100,100,0.5);"> Evaluate Your System on the Test Set </h2>

After tweaking your models, we can have a final model that performs well. Now is the time to evaluate the final model on the test set.

In [ ]:
final_model =  grid_search.best_estimator_

X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()

X_test_prepared = full_pipeline.transform(X_test)
final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)

print(final_rmse)

: 

In some cases, such a point estimate of the generalization error will not be quite enough, we can compute a 95% confidence interval for the generalization error using scipy.stats.t.interval():

In [ ]:
from scipy import stats
confidence = 0.95
squared_errors = (final_predictions - y_test) ** 2
np.sqrt(stats.t.interval(confidence, len(squared_errors) - 1,
                          loc=squared_errors.mean(),
                          scale=stats.sem(squared_errors)))

: 

If there was a lot of hyperparameter tuning, the performance will usually be slightly worse than what you measured using cross-validation. But, it is not the case in this example.

<h2 style="background-color:rgba(100,100,100,0.5);"> Summary </h2>

This exercise demonstrates what a Machine Learning project looks like as well as showing you some of the tools you can use to train a system. Most of the work is in the data preparation step. This example shows the overall process and applies three or four simple algorithms to predict the median housing price value.